In [2]:
!pip install scikit-learn nltk transformers torch

In [3]:
import nltk
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/logan.ramos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Synthetic Data (NOTE: pending professor approval) 

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Positive examples (labeled as 1)
synthetic_non_examples = [
    "I love the weather today, it's so sunny!",
    "This new phone is amazing, it has so many cool features.",
    "The book was an absolute masterpiece, I couldn’t put it down.",
    "I finally got the promotion I’ve been working so hard for!",
    "The new software update really improved my laptop's performance.",
    "I feel really happy with the decision I made to move.",
    "The park is such a peaceful place to walk and relax.",
    "I just received a beautiful gift from a friend!",
    "This recipe turned out fantastic, I will definitely make it again!",
    "The vacation was incredible, I had the best time.",
    "I finally fixed my car after days of trouble, such a relief!",
    "This new workout routine has made me feel much stronger.",
    "My friend and I had an amazing time at the concert last night."
]

# Negative examples (labeled as 0)
synthetic_examples = [
    "The movie was so boring, I fell asleep halfway through.",
    "I can't believe how bad the service was at that restaurant.",
    "This is the worst concert I’ve ever attended.",
    "The game was a huge disappointment, it wasn’t fun at all.",
    "The news was so depressing, I couldn’t watch it anymore.",
    "I’m really angry that I missed the meeting.",
    "I can’t believe how terrible the traffic was today.",
    "I had the worst customer service experience today.",
    "I was so disappointed by the product’s quality.",
    "I don't think I'll ever go back to that store again.",
    "I’m so frustrated with how slow the internet is.",
    "I regret buying that gadget, it was a waste of money."
]

positive_labels = [1] * len(synthetic_examples)
negative_labels = [0] * len(synthetic_non_examples)
labels = [*positive_labels, *negative_labels]
texts = [*synthetic_examples, *synthetic_non_examples]
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def encode_texts(texts):
    if isinstance(texts, str):
        texts = [texts]
    return tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
train_encodings = encode_texts(X_train)
test_encodings = encode_texts(X_test)
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)
train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
test_dataset = TensorDataset(test_encodings.input_ids, test_encodings.attention_mask, test_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
model.train()
epochs = 3
for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}/{epochs} completed.")
model.save_pretrained("straw_man_model")
tokenizer.save_pretrained("straw_man_model")

def predict_straw_man(text):
    encoding = encode_texts([text])
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()
    return "Strawman" if prediction == 1 else None

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 completed.
Epoch 2/3 completed.
Epoch 3/3 completed.


In [6]:
trump_zelensky = [
    "Vance: For four years, the United States of America, we had a president who stood up at press conferences and talked tough about Vladimir Putin, and then Putin invaded Ukraine and destroyed a significant chunk of the country. The path to peace and the path to prosperity is, maybe, engaging in diplomacy. We tried the pathway of Joe Biden, of thumping our chest and pretending that the president of the United States' words mattered more than the president of the United States' actions. What makes America a good country is America engaging in diplomacy. That's what President Trump is doing.",
    "Zelenskyy: Can I ask you?",
    "Vance: Sure. Yeah.",
    "Zelenskyy: OK. So he (Putin) occupied it, our parts, big parts of Ukraine, parts of east and Crimea. So he occupied it in 2014. So during a lot of years — I’m not speaking about just Biden, but those times was (Barack) Obama, then President Obama, then President Trump, then President Biden, now President Trump. And God bless, now, President Trump will stop him. But during 2014, nobody stopped him. He just occupied and took. He killed people. You know what the --",
    "Trump: 2015?",
    "Zelenskyy: 2014.",
    "Trump: Oh, 2014? I was not here.",
    "Vance: That’s exactly right.",
    "Zelenskyy: Yes, but during 2014 ‘til 2022, the situation is the same, that people have been dying on the contact line. Nobody stopped him. You know that we had conversations with him, a lot of conversations, my bilateral conversation. And we signed with him, me, like, you, president, in 2019, I signed with him the deal. I signed with him, (French President Emmanuel) Macron and (former German Chancellor Angela) Merkel. We signed ceasefire. Ceasefire. All of them told me that he will never go … But after that, he broke the ceasefire, he killed our people, and he didn’t exchange prisoners. We signed the exchange of prisoners. But he didn’t do it. What kind of diplomacy, JD, you are speaking about? What do you mean?"
]

# Dummy predictions for demonstration (replace with actual prediction function calls)
def predict_straw_man(s):
    return 'Strawman detected' if 'Trump' in s else None

def predict_ad_hominem(s):
    return 'Ad Hominem detected' if 'Zelenskyy' in s else None

for s in trump_zelensky:
    print(s)
    a = predict_straw_man(s)
    b = predict_ad_hominem(s)
    
    if a:
        print(a)
    if b:
        print(b)
    elif not a:
        print("nice!")

    print(f"-------\n")


Vance: For four years, the United States of America, we had a president who stood up at press conferences and talked tough about Vladimir Putin, and then Putin invaded Ukraine and destroyed a significant chunk of the country. The path to peace and the path to prosperity is, maybe, engaging in diplomacy. We tried the pathway of Joe Biden, of thumping our chest and pretending that the president of the United States' words mattered more than the president of the United States' actions. What makes America a good country is America engaging in diplomacy. That's what President Trump is doing.
Strawman detected
-------

Zelenskyy: Can I ask you?
Ad Hominem detected
-------

Vance: Sure. Yeah.
nice!
-------

Zelenskyy: OK. So he (Putin) occupied it, our parts, big parts of Ukraine, parts of east and Crimea. So he occupied it in 2014. So during a lot of years — I’m not speaking about just Biden, but those times was (Barack) Obama, then President Obama, then President Trump, then President Biden